In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import get_linear_schedule_with_warmup, AdamW

In [2]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import LightningModule

In [3]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from sklearn.metrics import mean_squared_error

In [4]:
train_ = pd.read_csv('train.csv')

In [5]:
train_hf = load_dataset("csv", data_files="train.csv")

Using custom data configuration default-70231d1f5162efc6
Found cached dataset csv (/Users/stanislavilyushin/.cache/huggingface/datasets/csv/default-70231d1f5162efc6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
data = train_hf['train'].train_test_split(train_size=0.9, seed = 42)
data['validation'] = data.pop('test')

Loading cached split indices for dataset at /Users/stanislavilyushin/.cache/huggingface/datasets/csv/default-70231d1f5162efc6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-7b41a49c0b05c9f8.arrow and /Users/stanislavilyushin/.cache/huggingface/datasets/csv/default-70231d1f5162efc6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-d25ed4bd47c65c05.arrow


In [7]:
model_name = "microsoft/deberta-v3-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/anaconda3/envs/transformers/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [8]:
target_cols = ['cohesion', 'syntax','vocabulary','phraseology','grammar','conventions']

MAX_LENGTH = 512
def preprocess_function(examples):
    result = tokenizer(
        examples['full_text'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True)
    
    if any(col in examples for col in target_cols):
        labels = list(zip(*[examples[col] for col in target_cols]))
        return {**result, "labels": labels}
    else:
        return result

data_preprocessed = data.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

In [10]:
class RMSELoss(nn.Module):
    def __init__(self, eps = 1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction = 'none')
        self.eps = eps
        
    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        loss = loss.mean()
        return loss

In [11]:
class ModelTuningUnfreeze(LightningModule):
    def __init__(self, model_name = "microsoft/deberta-v3-base"):
        super().__init__()
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.bert = transformers.AutoModel.from_pretrained(model_name)
        
        for param in self.bert.parameters():
            param.requires_grad=False  
            
        self.cnn = nn.Sequential(nn.Conv2d(in_channels=1, out_channels= 128,
                                           kernel_size= (512,768),stride = 2, padding = 1),
                                nn.MaxPool2d(kernel_size=2, stride=2, padding = 1),
                                nn.Flatten(),
                                nn.Dropout(0.2),
                                nn.ReLU())
        
        self.last = nn.Sequential(
        nn.Linear(1024,256),
            nn.GELU(),
        nn.Linear(256,6))
        self.criterion =  RMSELoss()
        
        #cnt_params = 0
        #for _ in self.bert.parameters():
            #cnt_params += 1
        
        #for i, param in enumerate(self.bert.parameters()):
            #if i < cnt_params - unfreeze_last:
                #param.requires_grad = False

        
    

    def forward(self, batch):
        bert_output = self.bert(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )['last_hidden_state'] # # (batch_size, dim_tokens, dim_bert)
        
        
        # bert_output = torch.max(bert_output, dim=1).values
        bert_output = torch.transpose(bert_output.unsqueeze(0), 0, 1) # (batch_size, 1, dim_tokens, dim_bert)
        
        bert_output = self.cnn(bert_output)

        return self.last(bert_output).squeeze(1)
    
    def training_step(self, batch, *args):
        pred = self.forward(batch)
        loss = self.criterion(pred, batch['labels'].to(device) + 0.0)
        self.log("training_loss", loss.item())
        return loss

    @torch.no_grad()
    def validation_step(self, batch, *args):
        pred = self.forward(batch)
        loss = self.criterion(pred, batch['labels'].to(device) + 0.0)
        self.log("val_loss", loss.item())
        return loss
    
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(self.parameters(), lr=3e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size = 10, gamma = 0.1)
        return {"optimizer": optimizer, 
               "lr_scheduler": {
                   "scheduler": scheduler,
                   "monitor": "training_loss"
               }}

In [12]:
model = ModelTuningUnfreeze().to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical 

In [13]:
train_loader = torch.utils.data.DataLoader(data_preprocessed['train'], batch_size= 16, 
                                           collate_fn=transformers.default_data_collator, num_workers = 8)

valid_loader = torch.utils.data.DataLoader(data_preprocessed['validation'], batch_size=16, 
                                           collate_fn=transformers.default_data_collator, num_workers = 8)

In [14]:
trainer = Trainer(max_epochs=10, gpus=torch.cuda.device_count(), gradient_clip_val=0.1)
trainer.fit(model, train_loader, valid_loader)

/opt/anaconda3/envs/transformers/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type           | Params
---------------------------------------------
0 | bert      | DebertaV2Model | 183 M 
1 | cnn       | Sequential     | 100 M 
2 | last      | Sequential     | 263 K 
3 | criterion | RMSELoss       | 0     
---------------------------------------------
100 M     Trainable params
183 M     Non-trainable params
284 M     Total params
1,139.036 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Training: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

Validation: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validation: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

`Trainer.fit` stopped: `max_epochs=10` reached.


In [15]:
torch.save(model.state_dict(), 'kaggle_model_base.pt')